# Sc2ts recombinant validation by external methods


In [1]:
import pandas as pd
import numpy as np

In [2]:
df_recomb = pd.read_csv("../data/recombinants.csv").set_index("sample_id")


In [3]:
# Add CovRecomb data
df = pd.read_csv("../arg_postprocessing/sc2ts_v1_2023-02-21_pr_pp_mp_aph_bps_pango_dated_CovRecomb_output.csv")
df = df.set_index("sample_id")

df_recomb["is_cov_recomb_recombinant"] = {sample_id: sample_id in df.index for sample_id in df_recomb.index}

In [4]:
df = pd.read_csv("../arg_postprocessing/sc2ts_v1_2023-02-21_pr_pp_mp_aph_bps_pango_dated_3seq.csv")
# Just getting yes/no answer for now
recombinants = df["recombinant"].unique()
df_recomb["is_3seq_recombinant"] = {k: row["recombinant"] in recombinants for k, row in df_recomb.iterrows()}


In [5]:
df_recomb["pass_qc"] = df_recomb["net_min_supporting_loci_lft_rgt_ge_4"]

In [6]:
df_summary = df_recomb[["pass_qc", "is_3seq_recombinant", "is_cov_recomb_recombinant", "is_rebar_recombinant"]]
df_summary

,pass_qc,is_3seq_recombinant,is_cov_recomb_recombinant,is_rebar_recombinant
sample_id,,,,
ERR9939974,False,False,False,False
SRR20259474,False,True,False,False
ERR10219711,False,False,True,False
ERR9848224,False,False,False,False
ERR9940192,False,True,False,False
...,...,...,...,...
SRR21797337,False,True,False,False
SRR21794908,False,True,False,False
ERR10708209,False,True,False,False


In [7]:
totals = df_summary.groupby("pass_qc").sum()
totals

,is_3seq_recombinant,is_cov_recomb_recombinant,is_rebar_recombinant
pass_qc,,,
False,146,2,10
True,338,90,203


In [8]:
denom = df_summary["pass_qc"].value_counts()
denom

pass_qc
False    501
True     354
Name: count, dtype: int64

In [9]:
totals["rebar_pc"] = np.round(100 * totals['is_rebar_recombinant'] / denom)
totals["3seq_pc"] = np.round(100 * totals["is_3seq_recombinant"] / denom)
totals["cov_recomb_pc"] = np.round(100 * totals["is_cov_recomb_recombinant"] / denom)
totals

,is_3seq_recombinant,is_cov_recomb_recombinant,is_rebar_recombinant,rebar_pc,3seq_pc,cov_recomb_pc
pass_qc,,,,,,
False,146,2,10,2.0,29.0,0.0
True,338,90,203,57.0,95.0,25.0


In [10]:
np.sum(df_recomb["is_3seq_recombinant"])

np.int64(484)

In [13]:
np.round(100 * np.sum(df_recomb["is_3seq_recombinant"]) / df_recomb.shape[0])

np.float64(57.0)

In [12]:
totals.sum()

is_3seq_recombinant          484.0
is_cov_recomb_recombinant     92.0
is_rebar_recombinant         213.0
rebar_pc                      59.0
3seq_pc                      124.0
cov_recomb_pc                 25.0
dtype: float64